### Imports and global utils

In [1]:
'''imports'''
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1,4,5,6,7"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pickle
# clear GPU memory
from utils import general_utils, eoc
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer, T5Tokenizer
from tqdm import tqdm
from datasets import load_dataset
from typing import Tuple, List
import torch.nn.functional as F

/home/nus-ytj/miniconda3/envs/inconsistencies/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load model

In [2]:
# We are using custom huggingface cache dirs in case the default one doesn't have the capacity, since the models can be quite large.
MY_HUGGINGFACE_CACHE_DIR ='huggingface_cache' # relative to this notebook path
tokenizer = AutoTokenizer.from_pretrained("google/ul2",
                                        cache_dir = MY_HUGGINGFACE_CACHE_DIR+'/google-ul2')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
RUN_CELL = False   # Load model
if RUN_CELL:
    model = T5ForConditionalGeneration.from_pretrained("google/ul2",
                                                        cache_dir=MY_HUGGINGFACE_CACHE_DIR + '/google-ul2',
                                                        low_cpu_mem_usage=True,
                                                        torch_dtype=torch.bfloat16,
                                                        device_map='balanced')

In [4]:
UL2_MODE = '[NLG]' # '[S2S]' is not supported by some functions (poor accuracy)

### Load dataset and specify partition

In [5]:
DATASET_PATH = "Rowan/hellaswag" # NOTE: hellaswag only has the validation set

# validation lens
# inputs max: 130, min: 19, avg: 66.98307110137422
# completions max: 125, min: 7, avg: 45.010655247958574
IS_DEVELOPMENT = True
data = load_dataset(DATASET_PATH)
set_partition = 'validation' if IS_DEVELOPMENT else 'test' 

RUN_CELL = True # shuffle data['validation']
if RUN_CELL:
    data['validation'] = data['validation'].shuffle(seed=42)
    # check sha256 to make sure shuffled data is the same
    # bd403791b54a53d53a955db1ef38500eb6b7ad4fe29e1164e8d455e82a529ad2
# only use 1000 samples for development
if IS_DEVELOPMENT:
    data['validation'] = data['validation'].select(range(1000))
sha256 = general_utils.hash_object(data['validation'][:].__str__())


Serialized object size: 1120628 bytes
SHA-256 hash: 88d4b0c37bcbc85630394178bcf73a543cef1f568c572dcf929decc48ac6d7f4


In [6]:
def get_dataset(
    set_partition: str,
    shuffle: bool = False,
    first_k_instances: int = None,
    seed = 42
):
    DATASET_PATH = "Rowan/hellaswag" # NOTE: hellaswag only has the validation set
    # validation lens
    # inputs max: 130, min: 19, avg: 66.98307110137422
    # completions max: 125, min: 7, avg: 45.010655247958574
    data = load_dataset(DATASET_PATH)
    if shuffle:
        data[set_partition] = data[set_partition].shuffle(seed)
    # bd403791b54a53d53a955db1ef38500eb6b7ad4fe29e1164e8d455e82a529ad2
    if first_k_instances is not None:
        data[set_partition] = data[set_partition].select(range(first_k_instances))
    if shuffle:
        general_utils.hash_object(data[set_partition][:].__str__())
    return data[set_partition]

In [7]:
data2 = get_dataset(set_partition='validation', shuffle=True, first_k_instances=1000, seed=42)

Serialized object size: 1120628 bytes
SHA-256 hash: 88d4b0c37bcbc85630394178bcf73a543cef1f568c572dcf929decc48ac6d7f4


In [33]:
data2[:]sys.getsizeof(serialized_obj)


{'ind': [33252,
  30637,
  46080,
  44754,
  9310,
  43612,
  41168,
  7515,
  38018,
  41506,
  12295,
  45596,
  33855,
  25503,
  28002,
  25653,
  38602,
  15529,
  24118,
  10359,
  27457,
  46372,
  8551,
  46888,
  16134,
  25645,
  4009,
  19077,
  30329,
  16931,
  492,
  15641,
  15208,
  3799,
  34173,
  20091,
  30508,
  13712,
  50272,
  124,
  17129,
  31883,
  4909,
  16338,
  43010,
  22366,
  40364,
  46674,
  28797,
  2582,
  34034,
  46261,
  19057,
  33374,
  40788,
  46359,
  22816,
  16825,
  39659,
  23308,
  16594,
  43085,
  17646,
  36240,
  4448,
  40207,
  47285,
  29715,
  21278,
  20143,
  40714,
  42888,
  18395,
  47387,
  22626,
  43847,
  8926,
  47629,
  2585,
  36031,
  43460,
  24980,
  45100,
  15352,
  35229,
  10448,
  34069,
  47984,
  12442,
  31975,
  11243,
  183,
  8301,
  3537,
  33256,
  42611,
  13617,
  49085,
  30942,
  46079,
  29073,
  43672,
  10512,
  31168,
  6829,
  32357,
  5861,
  50375,
  41827,
  29961,
  32588,
  47201,
  811

In [27]:
general_utils.hash_object(data['validation'])

'f2052acd022669e81a21c3cf2dd787b1fb6723e8e71b442574231fde8be43a17'

In [28]:
general_utils.hash_object(data2)

'f2052acd022669e81a21c3cf2dd787b1fb6723e8e71b442574231fde8be43a17'

### Define Loss Function

In [21]:
# define loss and get extra ids
ce_loss = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id) #reduction='avg'
ce_loss_sum = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id, reduction='sum') #reduction='sum'
extra_id_0 = torch.tensor([tokenizer.convert_tokens_to_ids("<extra_id_0>")])
extra_id_1 = torch.tensor([tokenizer.convert_tokens_to_ids("<extra_id_1>")])

### Define example generator

In [22]:
RUN_CELL = False
show = print if RUN_CELL else lambda *args, **kwargs: None


In [23]:
def example_generator(docs, tokenizer) -> Tuple:
    '''
        docs: DATA_SET[SUBJECTS_NAME], ex:HELLASWG_DATAS[validation]
        return: Tuple(input_ids, completions_ids_padded, labels)

        input[example]: ctx:<prompt> 
        label[example]: endings list -> list[] 

        Todo: few-shot data prompting
    '''
    example_id = 0
    for doc in docs:
        endings_list = doc['endings']
        input_ = UL2_MODE + " " + doc["activity_label"] + ": " + doc['ctx'] + " " + "<extra_id_0>"
        if UL2_MODE == "[NLG]":
            completions = [f"<extra_id_0> {ending}" for ending in endings_list]
        elif UL2_MODE == "[S2S]":
            completions = [f"{ending}" for ending in endings_list]
        
        # label = f"{endings_list[int(index)]}"
        show(input_)
        show(completions)

        input_ids = tokenizer(input_, return_tensors="pt").input_ids
        # label_id = tokenizer(label, return_tensors="pt").input_ids.to("cuda").clone().detach().requires_grad_(False)
        completions_ids = [tokenizer(completion, return_tensors="pt").input_ids[:,:-1]\
                                                                for completion in completions] # remove <eos> token with [:,:-1]

        # Assuming `max_length` is the maximum length you want to pad sequences to
        max_length = max(seq.size(1) for seq in completions_ids)

        # Pad sequences to the common length
        padded_sequences = [F.pad(seq, (0, max_length - seq.size(1)), value=tokenizer.pad_token_id) for seq in completions_ids]
        # Use pad_sequence
        completions_ids_padded = torch.nn.utils.rnn.pad_sequence(padded_sequences, batch_first=True, padding_value=tokenizer.pad_token_id)
        completions_ids_padded = torch.squeeze(completions_ids_padded, dim = 1)
        yield example_id, input_ids, completions_ids_padded, int(doc['label'])
        example_id += 1

In [24]:
RUN_CELL = True    # get input and completion lens
if RUN_CELL:
    input_lens = []
    completion_lens = []
    gen = example_generator(data[set_partition], tokenizer)
    for example_id, input_ids, completions_batch, label in tqdm(gen, total=len(data[set_partition])):
        input_lens.append(len(input_ids[0]))
        for completion in completions_batch:
            completion_lens.append(len(completion))
    print(f"input max: {max(input_lens)}, min: {min(input_lens)}, avg: {sum(input_lens)/len(input_lens)}")
    print(f"completion max: {max(completion_lens)}, min: {min(completion_lens)}, avg: {sum(completion_lens)/len(completion_lens)}")

100%|██████████| 1000/1000 [00:01<00:00, 934.09it/s]

input max: 123, min: 19, avg: 67.378
completion max: 85, min: 10, avg: 44.878


### Baseline

In [26]:
RUN_CELL = False   # generate baseline info and conditionals

if RUN_CELL:
    baseline = dict() 
    # save the label and the number of completions
    gen = example_generator(data[set_partition], tokenizer)
    for example_id, input_ids, completions_batch, label in tqdm(gen, total=len(data[set_partition])):
        baseline[example_id] = dict()
        baseline[example_id]['label'] = label
        baseline[example_id]['no_completions'] = len(completions_batch)
        baseline[example_id]['p_map'] = []
        p_and_completion = []

        outputs = eoc.multi_labels_forward(model, input_ids.cuda(), completions_batch.cuda())

        for completion_index in range(len(completions_batch)):
            if UL2_MODE == "[NLG]":            
                p = -ce_loss(
                    # Only care about the tokens corresponding to the last word and omit offset tokens 
                    # the first one is <extra_id_0> and omitted
                    outputs.logits[completion_index][1:], 
                    completions_batch[completion_index][1:].cuda()
                )
            elif UL2_MODE == "[S2S]":
                p = -ce_loss(
                    outputs.logits[completion_index], 
                    completions_batch[completion_index].cuda()
                )

            baseline[example_id]['p_map'] += [p.detach().cpu().tolist()]


100%|██████████| 1000/1000 [01:50<00:00,  9.09it/s]


In [27]:
RUN_CELL = False   # Save baseline
if RUN_CELL:
    pickle.dump(baseline, open("data/pkls/hellaswg/ul2/baseline_1000.pkl", "wb"))

In [29]:
RUN_CELL = True   # Load baseline
if RUN_CELL:
    '''Load the baseline p_maps'''
    with open("data/pkls/hellaswg/ul2/baseline_1000.pkl", "rb") as handle:
        baseline = pickle.load(handle)

### K-offset Conditionals

In [25]:
RUN_CELL = True 
if RUN_CELL:
    MAX_OFFSET = 3
    p_map_offset = dict() # maps (example_id, offset, completion_index) -> p_map
    for offset in range(1, MAX_OFFSET+1):
        gen = example_generator(data[set_partition], tokenizer)
        for example_id, input_ids, completions_batch, label in tqdm(gen, total=len(data[set_partition])):
            input_ids_offset, labels_offset = eoc.create_offset_sample_from_batch(
                tokenizer,
                input_ids,
                completions_batch,
                offset
            )
            outputs = eoc.multi_labels_forward(model, input_ids_offset.cuda(), labels_offset.cuda())
            for completion_index in range(len(completions_batch)):
                avg_log_p = -ce_loss(
                    # Only care about the tokens corresponding to the original completion and omit offset tokens 
                    # the first one is <extra_id_0> and omitted
                    outputs.logits[completion_index][1+offset:], 
                    labels_offset[completion_index][1+offset:].cuda()
                )
                p_map_offset[(example_id, offset, completion_index)] = \
                    avg_log_p.detach().cpu().tolist()

100%|██████████| 1000/1000 [01:49<00:00,  9.14it/s]


### Multispan Conditionals

In [ ]:
RUN_CELL = False   # generate multispan conditionals
if RUN_CELL:
    if UL2_MODE != "[NLG]":
        raise ValueError("Only NLG mode is supported for multispan conditionals for now")
    span_length = 1
    gap_between_spans = 1
    num_spans = 1
    p_map_multispan = dict()
    gen = example_generator(data[set_partition], tokenizer)

    for example_id, input_ids, completions_batch, label in tqdm(gen, total=len(data[set_partition])):
        # print(input_ids.shape)
        # continue
        inputs_ids_multispan, labels_multispan = eoc.create_multiple_span_sample_from_batch(
            tokenizer,
            input_ids[0], # squeeze 1st dim
            completions_batch,
            span_length,
            gap_between_spans,
            num_spans,
        )
        outputs = eoc.multi_labels_forward(model, inputs_ids_multispan.cuda(), labels_multispan.cuda())

        # assert multispan samples are correct 
        assert completions_batch[completion_index].nonzero().shape[0] == \
            labels_multispan[completion_index][num_spans * (span_length + 1) :].nonzero().shape[0]

        for completion_index in range(len(completions_batch)):
            avg_log_p = -ce_loss(
                # Only care about the tokens corresponding to the completion (see assert below)); 
                # so the first <extra_id_0> is omitted, and for each span, the span + <extra_id_k> is omitted;
                # totally 1 + num_spans * (span_length + 1) tokens are omitted;
                # labels_multispan contains paddings.
                outputs.logits[completion_index][1 + num_spans * (span_length + 1) :], 
                labels_multispan[completion_index][1 + num_spans * (span_length + 1) :].cuda()
            )
            p_map_multispan[(example_id, span_length, gap_between_spans, num_spans, completion_index)] = \
                avg_log_p.detach().cpu().tolist()

### Ensemble of Conditionals

In [41]:
RUN_CELL = True    # Max reduction to emsemble conditionals for the same last word
'''Max reduction to emsemble conditionals, i.e., only the maximum avg_log_p is kept for each completion.
Emsemble the baseline conditionals with the K-offset conditionals and middle-off conditionals.'''

if RUN_CELL:
    # Add the baseline (offset = 0 from K-offset ensemble) to the list
    ADD_BASELINE = True

    # Add K-offset conditionals to the list
    ADD_K_OFFSET = False
    MAX_OFFSET = 3

    # Add multispan conditionals to the list
    ADD_MULTISPAN = False
    LENGTH_GAP_NUM_TUPLES = [
        (1, 1, 1), 
    ] # SPAN_LENGTH, GAP_LENGTH, NUM_SPANS. NUM_SPANS can be float, which is treated as auto_ratio.

    count_correct = 0
    for example_index in tqdm(range(len(data[set_partition]))): # len(lambada)
        no_completions = baseline[example_index]['no_completions']
        # Create a list of tuples (avg_log_p, completion) for each completion
        p_and_completion = []
        
        # add the baseline (offset = 0 from K-offset ensemble) to the list
        if ADD_BASELINE:
            p_and_completion += [
                (baseline[example_index]['p_map'][completion_index], completion_index)
                for completion_index in range(no_completions)
            ]
            
        # add the whole K-offset ensemble to the list
        if ADD_K_OFFSET:
            for offset in range(1, MAX_OFFSET + 1):
                p_and_completion += [
                    (p_map_offset[(example_index, offset, completion_index)], completion_index)
                    for completion_index in range(no_completions)
                ]
                
        if ADD_MULTISPAN:
            p_and_completion += [
                (p_map_multispan[(example_index, *length_gap_num, completion_index)], completion_index)
                for completion_index in range(no_completions)
                for length_gap_num in LENGTH_GAP_NUM_TUPLES
            ]

        # Find the tuple with the maximum avg_log_p; this is essentially max reduction
        _, best_completion_index = max(p_and_completion, key=lambda x: x[0])
        if best_completion_index == baseline[example_index]['label']:
            count_correct += 1
    print("accuracy:", count_correct / len(data[set_partition]))

100%|██████████| 1000/1000 [00:00<00:00, 365739.80it/s]

accuracy: 0.758


### Experiment Log

#### NLG
baseline: 0.758

3,3,1 works poorly 

3,5,1 works poorly: 0.732 (w/o baseline)

2,1,1 works poorly: 0.735 (w/o baseline)
    
1,1,1 works poorly: 0.729 (w/o baseline)

#### S2S
baseline: 0.712
